<a href="https://colab.research.google.com/github/quantam665/Loan-Management-System/blob/main/Loan_Management_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# ✅ FULL STREAMLIT APP IN A SINGLE CELL (Google Colab)
!pip install streamlit pyngrok pandas openpyxl --quiet
!ngrok config add-authtoken 2yAxXlXEyCxUXbQWe3O1nBydZTp_TDQzgdjZLwopBAa6qiFZ

from pyngrok import ngrok
import threading

# ✅ Step 3: Write Streamlit app to a file
with open("app.py", "w") as f:
    f.write('''import streamlit as st
import pandas as pd
from datetime import datetime, timedelta

st.set_page_config(page_title="Loan Approval App", page_icon="💰")

st.sidebar.title("📁 Upload Files")
customer_file = st.sidebar.file_uploader("Upload customer_data.xlsx", type=["xlsx"])
loan_file = st.sidebar.file_uploader("Upload loan_data.xlsx", type=["xlsx"])

if customer_file and loan_file:
    customer_df = pd.read_excel(customer_file)
    loan_df = pd.read_excel(loan_file)

    customer_df.columns = customer_df.columns.str.lower().str.replace(" ", "_")
    loan_df.columns = loan_df.columns.str.lower().str.replace(" ", "_")
    loan_df.rename(columns={"monthly_payment": "monthly_repayment", "date_of_approval": "start_date"}, inplace=True)

    # ✅ Fix datetime conversion and handle NaT values
    loan_df["start_date"] = pd.to_datetime(loan_df["start_date"], errors="coerce")

    # ✅ Drop rows with invalid dates or fill with default date
    loan_df = loan_df.dropna(subset=["start_date"])
    # Alternative: loan_df["start_date"] = loan_df["start_date"].fillna(pd.Timestamp("2020-01-01"))

    if "current_debt" not in customer_df.columns:
        customer_df["current_debt"] = 0

    def calculate_emi(principal, rate, tenure):
        r = rate / (12 * 100)
        return round(principal * r * (1 + r)**tenure / ((1 + r)**tenure - 1), 2)

    st.title("🏦 Loan Management System")

    customer_id = st.number_input("Enter Customer ID", min_value=1, value=1)

    if st.button("📄 Show Customer Profile"):
        profile = customer_df[customer_df["customer_id"] == customer_id]
        if not profile.empty:
            st.write(profile)
        else:
            st.error("Customer not found.")

    st.subheader("💰 Request New Loan")
    amount = st.number_input("Loan Amount", min_value=1000)
    rate = st.number_input("Interest Rate (%)", min_value=0.0, value=10.0)
    months = st.number_input("Tenure (months)", min_value=1)

    if st.button("✅ Approve Loan"):
        profile = customer_df[customer_df["customer_id"] == customer_id]
        if profile.empty:
            st.error("Customer not found")
        else:
            profile = profile.squeeze()
            emi = calculate_emi(amount, rate, months)
            if emi + profile["current_debt"] > 0.5 * profile["monthly_salary"]:
                st.warning("❌ Rejected. EMI exceeds 50% of salary.")
            else:
                loan_id = f"LN{1000 + len(loan_df)}"
                today = datetime.today().date()
                end = today + timedelta(days=30 * months)
                # ✅ Create new row as dict to avoid pandas warnings
                new_loan = {
                    "customer_id": customer_id,
                    "loan_id": loan_id,
                    "loan_amount": amount,
                    "tenure": months,
                    "interest_rate": rate,
                    "monthly_repayment": emi,
                    "emis_paid_on_time": 0,
                    "start_date": pd.Timestamp(today),
                    "end_date": pd.Timestamp(end)
                }
                loan_df = pd.concat([loan_df, pd.DataFrame([new_loan])], ignore_index=True)
                st.success(f"✅ Loan Approved! EMI: ₹{emi}/month")

    st.subheader("📚 Loan History with Filters")

    # ✅ Only show filters if we have loan data
    if not loan_df.empty:
        min_emi = st.slider("Min EMI", 0, 100000, 0)
        max_emi = st.slider("Max EMI", 0, 100000, 100000)
        start_date = st.date_input("Start Date Filter", datetime(2000, 1, 1))
        end_date = st.date_input("End Date Filter", datetime.today())

        start_ts = pd.Timestamp(start_date)
        end_ts = pd.Timestamp(end_date)

        # ✅ Safe filtering with proper datetime handling
        customer_loans = loan_df[loan_df["customer_id"] == customer_id]

        if not customer_loans.empty:
            filtered = customer_loans[
                (customer_loans["monthly_repayment"] >= min_emi) &
                (customer_loans["monthly_repayment"] <= max_emi) &
                (customer_loans["start_date"] >= start_ts) &
                (customer_loans["start_date"] <= end_ts)
            ]

            if not filtered.empty:
                st.write(filtered)
            else:
                st.info("No loans found matching the filters.")
        else:
            st.info("No loans found for this customer.")
    else:
        st.info("No loan data available.")

else:
    st.warning("⬆️ Please upload both Excel files to begin.")
''')

# ✅ Step 4: Run Streamlit app in background
def run_app():
    !streamlit run app.py --server.port 8501 --server.enableCORS false > /dev/null 2>&1

threading.Thread(target=run_app).start()

# ✅ Step 5: Open public ngrok tunnel
ngrok.kill()
public_url = ngrok.connect(8501)
print(f"🚀 Your app is live!\n👉 {public_url}")

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
🚀 Your app is live!
👉 NgrokTunnel: "https://4709505db3eb.ngrok-free.app" -> "http://localhost:8501"
